In [ ]:
import pandas as pd
from collections import deque, defaultdict
import heapq
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
path_candidates='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Candidates.csv'
path_seats='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Seats.csv'
# path_seatsFor='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Seats_Foreign.csv'
path_normChoices='/home/it/Desktop/RUN/Official/CSAB_SuperNumerary/Normal_Virtualized_Choices.csv'
path_dsChoices='/home/it/Desktop/RUN/Official/CSAB_SuperNumerary/DS_Virtualized_Choices.csv'
path_choices='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Choice.csv'
path_allotment='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Allotment_NIC.csv'
path_output='allotment_NITK.csv'
path_error='allotment_NITK_unalloted.csv'

In [3]:
class Program:
    def __init__(self, inst_code, brcode, quota, cat, subcat, gender, capacity, ds, min_cutoff=0, round_no=1):
        self.inst_code = inst_code
        self.brcode = brcode
        self.capacity = capacity
        self.gender = gender
        self.quota = quota
        self.cat = cat
        self.subcat = subcat
        self.ds=ds
        self.min_cutoff = min_cutoff
        self.round_no = round_no
        self.waitlist = []

    def is_eligible(self, rk):
        if self.round_no == 1:
            return True
        return rk <= self.min_cutoff

    def add_candidate(self, rk, roll_no):
        if rk>0:
            heapq.heappush(self.waitlist, (-rk, roll_no))
            return True
        return False

    def remove_least_preferred(self):
        return heapq.heappop(self.waitlist)

class Candidate:
    def __init__(self, roll_no, 
                 AI_Eng_CRL_Rank, AI_Eng_OBC_NCL_Rank, AI_Eng_SC_Rank, AI_Eng_ST_Rank, AI_Eng_EWS_Rank, 
                 AI_Eng_CRL_PD_Rank, AI_Eng_OBC_NCL_PD_Rank, AI_Eng_SC_PD_Rank, AI_Eng_ST_PD_Rank, AI_Eng_EWS_PD_Rank, 
                 AI_ARC_CRL_Rank, AI_ARC_OBC_NCL_Rank, AI_ARC_SC_Rank, AI_ARC_ST_Rank, AI_ARC_EWS_Rank, 
                 AI_ARC_CRL_PD_Rank, AI_ARC_OBC_NCL_PD_Rank, AI_ARC_SC_PD_Rank, AI_ARC_ST_PD_Rank, AI_ARC_EWS_PD_Rank, 
                 AI_Pln_CRL_Rank, AI_Pln_OBC_NCL_Rank, AI_Pln_SC_Rank, AI_Pln_ST_Rank, AI_Pln_EWS_Rank, 
                 AI_Pln_CRL_PD_Rank, AI_Pln_OBC_NCL_PD_Rank, AI_Pln_SC_PD_Rank, AI_Pln_ST_PD_Rank, AI_Pln_EWS_PD_Rank, 
                 Adv_CRL_Rank, Adv_OBC_NCL_Rank, Adv_SC_Rank, Adv_ST_Rank, Adv_EWS_Rank, 
                 Adv_CRL_PD_Rank, Adv_OBC_NCL_PD_Rank, Adv_SC_PD_Rank, Adv_ST_PD_Rank, Adv_EWS_PD_Rank, 
                 Adv_Prep_SC_Rank, Adv_Prep_ST_Rank, Adv_Prep_CRL_PD_Rank, Adv_Prep_OBC_NCL_PD_Rank, 
                 Adv_Prep_SC_PD_Rank, Adv_Prep_ST_PD_Rank, Adv_Prep_EWS_PD_Rank, Adv_DS, prep):

        self.roll_no = roll_no
        self.current_index = 1
        self.max_index = 0
        self.choice_list = {}
        self.choice_list_mapper = {}
        if Adv_DS!=1:
                Adv_DS=0
        self.isDS=Adv_DS
        self.prep=prep

        self.AI_Eng_CRL_Rank = AI_Eng_CRL_Rank
        self.AI_Eng_OBC_NCL_Rank = AI_Eng_OBC_NCL_Rank
        self.AI_Eng_SC_Rank = AI_Eng_SC_Rank
        self.AI_Eng_ST_Rank = AI_Eng_ST_Rank
        self.AI_Eng_EWS_Rank = AI_Eng_EWS_Rank
        self.AI_Eng_CRL_PD_Rank = AI_Eng_CRL_PD_Rank
        self.AI_Eng_OBC_NCL_PD_Rank = AI_Eng_OBC_NCL_PD_Rank
        self.AI_Eng_SC_PD_Rank = AI_Eng_SC_PD_Rank
        self.AI_Eng_ST_PD_Rank = AI_Eng_ST_PD_Rank
        self.AI_Eng_EWS_PD_Rank = AI_Eng_EWS_PD_Rank
        
        self.AI_ARC_CRL_Rank = AI_ARC_CRL_Rank
        self.AI_ARC_OBC_NCL_Rank = AI_ARC_OBC_NCL_Rank
        self.AI_ARC_SC_Rank = AI_ARC_SC_Rank
        self.AI_ARC_ST_Rank = AI_ARC_ST_Rank
        self.AI_ARC_EWS_Rank = AI_ARC_EWS_Rank
        self.AI_ARC_CRL_PD_Rank = AI_ARC_CRL_PD_Rank
        self.AI_ARC_OBC_NCL_PD_Rank = AI_ARC_OBC_NCL_PD_Rank
        self.AI_ARC_SC_PD_Rank = AI_ARC_SC_PD_Rank
        self.AI_ARC_ST_PD_Rank = AI_ARC_ST_PD_Rank
        self.AI_ARC_EWS_PD_Rank = AI_ARC_EWS_PD_Rank
        
        self.AI_Pln_CRL_Rank = AI_Pln_CRL_Rank
        self.AI_Pln_OBC_NCL_Rank = AI_Pln_OBC_NCL_Rank
        self.AI_Pln_SC_Rank = AI_Pln_SC_Rank
        self.AI_Pln_ST_Rank = AI_Pln_ST_Rank
        self.AI_Pln_EWS_Rank = AI_Pln_EWS_Rank
        self.AI_Pln_CRL_PD_Rank = AI_Pln_CRL_PD_Rank
        self.AI_Pln_OBC_NCL_PD_Rank = AI_Pln_OBC_NCL_PD_Rank
        self.AI_Pln_SC_PD_Rank = AI_Pln_SC_PD_Rank
        self.AI_Pln_ST_PD_Rank = AI_Pln_ST_PD_Rank
        self.AI_Pln_EWS_PD_Rank = AI_Pln_EWS_PD_Rank
        
        self.Adv_CRL_Rank = Adv_CRL_Rank
        self.Adv_OBC_NCL_Rank = Adv_OBC_NCL_Rank
        self.Adv_SC_Rank = Adv_SC_Rank
        self.Adv_ST_Rank = Adv_ST_Rank
        self.Adv_EWS_Rank = Adv_EWS_Rank
        self.Adv_CRL_PD_Rank = Adv_CRL_PD_Rank
        self.Adv_OBC_NCL_PD_Rank = Adv_OBC_NCL_PD_Rank
        self.Adv_SC_PD_Rank = Adv_SC_PD_Rank
        self.Adv_ST_PD_Rank = Adv_ST_PD_Rank
        self.Adv_EWS_PD_Rank = Adv_EWS_PD_Rank
        
        self.Adv_Prep_SC_Rank = Adv_Prep_SC_Rank
        self.Adv_Prep_ST_Rank = Adv_Prep_ST_Rank
        self.Adv_Prep_CRL_PD_Rank = Adv_Prep_CRL_PD_Rank
        self.Adv_Prep_OBC_NCL_PD_Rank = Adv_Prep_OBC_NCL_PD_Rank
        self.Adv_Prep_SC_PD_Rank = Adv_Prep_SC_PD_Rank
        self.Adv_Prep_ST_PD_Rank = Adv_Prep_ST_PD_Rank
        self.Adv_Prep_EWS_PD_Rank = Adv_Prep_EWS_PD_Rank

def load_candidates(filename):
    df = pd.read_csv(filename)
    filtered_df = df.sort_values('Adv_CRL_Rank')
    filtered_df = filtered_df[filtered_df['Nationality'] != 4]
    return {row['RollNo']: Candidate(row['RollNo']
      ,row['AI_Eng_CRL_Rank']  
      ,row['AI_Eng_OBC_NCL_Rank']
      ,row['AI_Eng_SC_Rank']
      ,row['AI_Eng_ST_Rank']
      ,row['AI_Eng_EWS_Rank']
      ,row['AI_Eng_CRL_PD_Rank']
      ,row['AI_Eng_OBC_NCL_PD_Rank']
      ,row['AI_Eng_SC_PD_Rank']
      ,row['AI_Eng_ST_PD_Rank']
      ,row['AI_Eng_EWS_PD_Rank']
      ,row['AI_ARC_CRL_Rank']
      ,row['AI_ARC_OBC_NCL_Rank']
      ,row['AI_ARC_SC_Rank']
      ,row['AI_ARC_ST_Rank']
      ,row['AI_ARC_EWS_Rank']
      ,row['AI_ARC_CRL_PD_Rank']
      ,row['AI_ARC_OBC_NCL_PD_Rank']
      ,row['AI_ARC_SC_PD_Rank']
      ,row['AI_ARC_ST_PD_Rank']
      ,row['AI_ARC_EWS_PD_Rank']
      ,row['AI_Pln_CRL_Rank']
      ,row['AI_Pln_OBC_NCL_Rank']
      ,row['AI_Pln_SC_Rank']
      ,row['AI_Pln_ST_Rank']
      ,row['AI_Pln_EWS_Rank']
      ,row['AI_Pln_CRL_PD_Rank']
      ,row['AI_Pln_OBC_NCL_PD_Rank']
      ,row['AI_Pln_SC_PD_Rank']
      ,row['AI_Pln_ST_PD_Rank']
      ,row['AI_Pln_EWS_PD_Rank']
      ,row['Adv_CRL_Rank']
      ,row['Adv_OBC_NCL_Rank']
      ,row['Adv_SC_Rank']
      ,row['Adv_ST_Rank']
      ,row['Adv_EWS_Rank']
      ,row['Adv_CRL_PD_Rank']
      ,row['Adv_OBC_NCL_PD_Rank']
      ,row['Adv_SC_PD_Rank']
      ,row['Adv_ST_PD_Rank']
      ,row['Adv_EWS_PD_Rank']
      ,row['Adv_Prep_SC_Rank']
      ,row['Adv_Prep_ST_Rank']
      ,row['Adv_Prep_CRL_PD_Rank']
      ,row['Adv_Prep_OBC_NCL_PD_Rank']
      ,row['Adv_Prep_SC_PD_Rank']
      ,row['Adv_Prep_ST_PD_Rank']
      ,row['Adv_Prep_EWS_PD_Rank'], row['Adv_DS'], row['Adv_IsPrep']) for index, row in filtered_df.iterrows()}

def load_programs(filename):
    df = pd.read_csv(filename)
    return {(row['InstCd'], row['BrCd'], row['Quota'], row['Cat'], row['SubCat'], row['Gender'], 0): Program(row['InstCd'], row['BrCd'], row['Quota'], row['Cat'], row['SubCat'], row['Gender'], row['TSeat'], 0) for index, row in df.iterrows()}

# Cat,SubCat,Gender,StCd1,StCd2,StCd3,StCd4,StCd5,StCd6,StCd7,StCd8,StCd9,StCd10,TSeat
candidates = load_candidates(path_candidates)
programs = load_programs(path_seats)
print("The easy part is done.")



In [ ]:
def process_chunk(chunk):
    local_candidates = {}
    for index, row in chunk.iterrows():
        roll_no = int(row['RollNo'])
        choice_no = row['ChoiceNo']
        v_choice_no = int(row['ChoiceNo_V'])
        inst_code = row['InstCode']
        brcode = row['Brcode']
        quota = row['State_quota_V']
        cat = row['Cat_V']
        subcat = row['SubCat_V']
        gender = row['Gender_V']
        if roll_no in candidates.keys():
            if roll_no not in local_candidates:
                local_candidates[roll_no] = {'choice_list': {}, 'choice_list_mapper': {}, 'max_index': 0}
            program_key = (inst_code, brcode, quota, cat, subcat, gender, 0)
            program = programs.get(program_key)
            local_candidates[roll_no]['choice_list'][v_choice_no - 1] = program
            local_candidates[roll_no]['choice_list_mapper'][v_choice_no - 1] = choice_no
            local_candidates[roll_no]['max_index'] += 1
    return local_candidates

def merge_results(results):
    for result in results:
        for roll_no, data in result.items():
            if roll_no in candidates:
                candidates[roll_no].choice_list.update(data['choice_list'])
                candidates[roll_no].choice_list_mapper.update(data['choice_list_mapper'])
                candidates[roll_no].max_index += data['max_index']

def load_choices_parallel(filename, chunk_size=500000):
    df = pd.read_csv(filename, chunksize=chunk_size)
    futures = []
    with ProcessPoolExecutor(max_workers=20) as executor:
        for chunk in df:
            futures.append(executor.submit(process_chunk, chunk))
        results = []
        for future in as_completed(futures):
            results.append(future.result())
    merge_results(results)

    
start_time = time.time()
load_choices_parallel(path_normChoices)     
end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken for loading choices in parallel: {elapsed_time:.2f} seconds")
print("The tough part is done.")


Time taken for loading choices in parallel: 177.36 seconds
The tough part is done.


In [ ]:
def ds_process_chunk(chunk):
    local_candidates = {}
    for index, row in chunk.iterrows():
        roll_no = int(row['RollNo'])
        choice_no = row['ChoiceNo']
        v_choice_no = int(row['ChoiceNo_V'])
        inst_code = row['InstCode']
        brcode = row['Brcode']
        quota = row['State_quota_V']
        cat = row['Cat_V']
        subcat = row['SubCat_V']
        gender = row['Gender_V']
        ds = 0
        if row['Adv_DS_V']==1 and inst_code<200:
            ds=1
            
        if roll_no in candidates.keys():
            if roll_no not in local_candidates:
                local_candidates[roll_no] = {'choice_list': {}, 'choice_list_mapper': {}, 'max_index': 0}

            program_key = (inst_code, brcode, quota, cat, subcat, gender, ds)
            if program_key not in programs:
                programs[program_key]=Program(inst_code, brcode, quota, cat, subcat, gender, 0, ds)
            program = programs.get(program_key)
            local_candidates[roll_no]['choice_list'][v_choice_no - 1] = program
            local_candidates[roll_no]['choice_list_mapper'][v_choice_no - 1] = choice_no
            local_candidates[roll_no]['max_index'] += 1
    return local_candidates

def ds_merge_results(results):
    for result in results:
        for roll_no, data in result.items():
            if roll_no in candidates:
                candidates[roll_no].choice_list.update(data['choice_list'])
                candidates[roll_no].choice_list_mapper.update(data['choice_list_mapper'])
                candidates[roll_no].max_index += data['max_index']

def ds_load_choices_parallel(filename, chunk_size=500000):
    df = pd.read_csv(filename, chunksize=chunk_size)
    futures = []
    with ProcessPoolExecutor(max_workers=20) as executor:
        for chunk in df:
            futures.append(executor.submit(ds_process_chunk, chunk))
        results = []
        for future in as_completed(futures):
            results.append(future.result())
    ds_merge_results(results)

    
start_time = time.time()
ds_load_choices_parallel(path_dsChoices)   
end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken for loading choices in parallel: {elapsed_time:.2f} seconds")
print("The tough part is done.")

Time taken for loading choices in parallel: 5.75 seconds
The tough part is done.


In [ ]:
def resetter():
    for program_key, _ in programs.items():
        programs[program_key].waitlist=[]
    for candidate_key, _ in candidates.items():
        candidates[candidate_key].current_index=1
    print("Successfully reset.")
resetter()
allocations={}
ds_inst=defaultdict(list)

Successfully reset.


In [ ]:
sups={}
totSeats={}
for program_key, program in programs.items():
    if program.capacity<len(program.waitlist):
        print(f"Supernumerary in {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {program_key[4]}, {program_key[5]} : {len(program.waitlist)-program.capacity} seats")
        if program_key[3] not in sups.keys():
            sups[program_key[3]]=0
        sups[program_key[3]]+=len(program.waitlist)-program.capacity
        if program_key[3]+program_key[4] not in totSeats.keys():
            totSeats[program_key[3]+program_key[4]]=0
        totSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
    else:
        if program_key[3]+program_key[4] not in totSeats.keys():
            totSeats[program_key[3]+program_key[4]]=0
        totSeats[program_key[3]+program_key[4]]+=program.capacity
               
print(sups)
print(totSeats)

In [ ]:
start_time = time.time()

def getRank(roll_no, program):
    candidate = candidates[roll_no]
    if program.inst_code<200:
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.Adv_CRL_Rank
            else:
                return candidate.Adv_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.Adv_OBC_NCL_Rank
            else:
                return candidate.Adv_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.Adv_SC_Rank
            else:
                return candidate.Adv_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.Adv_EWS_Rank
            else:
                return candidate.Adv_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.Adv_ST_Rank
            else:
                return candidate.Adv_ST_PD_Rank
    elif program.brcode == "4501":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Pln_CRL_Rank
            else:
                return candidate.AI_Pln_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Pln_OBC_NCL_Rank
            else:
                return candidate.AI_Pln_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Pln_SC_Rank
            else:
                return candidate.AI_Pln_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Pln_EWS_Rank
            else:
                return candidate.AI_Pln_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Pln_ST_Rank
            else:
                return candidate.AI_Pln_ST_PD_Rank  
            
    elif program.brcode == "5101" or program.brcode == "5102" or program.brcode == "5103":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_ARC_CRL_Rank
            else:
                return candidate.AI_ARC_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_ARC_OBC_NCL_Rank
            else:
                return candidate.AI_ARC_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_ARC_SC_Rank
            else:
                return candidate.AI_ARC_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_ARC_EWS_Rank
            else:
                return candidate.AI_ARC_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_ARC_ST_Rank
            else:
                return candidate.AI_ARC_ST_PD_Rank
    else:
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Eng_CRL_Rank
            else:
                return candidate.AI_Eng_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Eng_OBC_NCL_Rank
            else:
                return candidate.AI_Eng_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Eng_SC_Rank
            else:
                return candidate.AI_Eng_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Eng_EWS_Rank
            else:
                return candidate.AI_Eng_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Eng_ST_Rank
            else:
                return candidate.AI_Eng_ST_PD_Rank
        else:
            return -1

def process_candidates():
    start_time = time.time()
    allocations = {}
    while stack:
        candidate = stack.pop()
        if candidate.current_index<=candidate.max_index:
            preference = candidates[candidate.roll_no].choice_list[candidate.current_index-1]
            if preference :
                if preference.ds==1:
                    if not candidate.Adv_CRL_Rank>0:
                        candidates[candidate.roll_no].current_index += 1
                        stack.append(candidate)
                        continue
                    preference = programs[preference.inst_code, preference.brcode, preference.quota, preference.cat, preference.subcat, preference.gender, 0]
                    if len(ds_inst[preference.inst_code])<2:
                        ds_inst[preference.inst_code].append({'roll_no': candidate.roll_no, 'brcode': preference.brcode, 'rank': candidate.Adv_CRL_Rank})
                    else:
                        ds_institute=sorted([x for x in ds_inst[preference.inst_code]], key=lambda x: -x["rank"])
                        if ds_institute[0]['rank']<candidate.Adv_CRL_Rank:
                            candidates[candidate.roll_no].current_index += 1
                            stack.append(candidate)
                        else:
                            ds_inst[preference.inst_code] = [item for item in ds_inst[preference.inst_code] if item['roll_no'] != ds_institute[0]["roll_no"]]
                            ds_inst[preference.inst_code].append({'roll_no': candidate.roll_no, 'brcode': preference.brcode, 'rank': candidate.Adv_CRL_Rank})
                            candidates[ds_institute[0]['roll_no']].current_index += 1
                            stack.append(candidates[ds_institute[0]['roll_no']])
                else:
                    preference = programs[preference.inst_code, preference.brcode, preference.quota, preference.cat, preference.subcat, preference.gender, 0]
                    rk = getRank(candidate.roll_no, preference)
                    if preference.add_candidate(rk, candidate.roll_no):
                        allocations[candidate.roll_no] = preference
                        if len(preference.waitlist) > preference.capacity:
                            last_rank = -preference.waitlist[0][0]
                            if last_rank > preference.min_cutoff:
                                count_same_rank = sum(1 for x in preference.waitlist if -x[0] == last_rank)
                                if len(preference.waitlist) - count_same_rank >= preference.capacity:
                                    removed_candidates = []
                                    while preference.waitlist and -preference.waitlist[0][0] == last_rank:
                                        _, roll_no = heapq.heappop(preference.waitlist)
                                        removed_candidates.append(roll_no)
                                    for roll_no in removed_candidates:
                                        candidates[roll_no].current_index += 1 
                                        candidate = candidates[roll_no] 
                                        allocations[roll_no] = None
                                        stack.append(candidate)
                                else:
                                    print(f"At {preference.inst_code}, {preference.brcode}, {preference.quota}, {preference.cat}, {preference.subcat}, {preference.gender}, same rank {rk} - Extra {len(preference.waitlist) - count_same_rank - preference.capacity}")
                                    # programs[preference.inst_code, preference.brcode, preference.quota, preference.cat, preference.subcat, preference.gender, 0].capacity+=1
                    else:
                        candidates[candidate.roll_no].current_index += 1
                        stack.append(candidate)
                
    print(f"Time taken: {time.time() - start_time:.2f} seconds")
    return allocations, programs

ds_inst=defaultdict(list)
filtered_candidates = [c for c in candidates.values()]
filtered_candidates.sort(key=lambda c: (c.Adv_CRL_Rank == 0, c.AI_Eng_CRL_Rank > 0, c.Adv_CRL_Rank))
stack = []
stack.extend(filtered_candidates[::-1])
allocations, programs = process_candidates()
        
end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken for allocation: {elapsed_time:.2f} seconds")


In [ ]:
old_pgm=programs
reallocs={}
for program_key, program in programs.items():
    if program_key[4]=="PH" and program.capacity>len(program.waitlist) and program_key[0]<200:
        programs[program_key[0], program_key[1], program_key[2], program_key[3], "NO", program_key[5], 0].capacity+=program.capacity-len(program.waitlist)
        if program_key[3] not in reallocs.keys():
            reallocs[program_key[3]]=0
        reallocs[program_key[3]]+=program.capacity-len(program.waitlist)
        print(f"Shifted {program.capacity-len(program.waitlist)} from {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {"PH"}, {program_key[5]} to {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {"NO"}, {program_key[5]}")
        programs[program_key].capacity=len(program.waitlist)
        

In [ ]:
def ENDresetter():
    for program_key, _ in programs.items():
        programs[program_key].waitlist=[]
    for candidate_key, _ in candidates.items():
        candidates[candidate_key].current_index=1
    print("Successfully reset.")
ENDresetter()
allocations={}
ds_inst=defaultdict(list)
filtered_candidates = [c for c in candidates.values()]
filtered_candidates.sort(key=lambda c: (c.Adv_CRL_Rank == 0, c.AI_Eng_CRL_Rank > 0, c.Adv_CRL_Rank))
stack = []
stack.extend(filtered_candidates[::-1])
allocations, programs = process_candidates()        

In [ ]:
sum_ds=0
for inst_code, students in ds_inst.items():
    print(f"Institution Code: {inst_code}")
    sum_ds+=len(students)
    for student in students:
        print(f"  Roll No: {student['roll_no']}, Branch Code: {student['brcode']}, Rank: {student['rank']}")
print(sum_ds)

In [ ]:
print("Final Allocations:")
for roll_no, preference in allocations.items():
    candidate = candidates.get(roll_no)
    if preference:
        print(f"Candidate {roll_no} is allocated to program {preference.inst_code}'s {preference.brcode} with {preference.quota}, {preference.cat}, {preference.subcat}, {preference.gender} : Mains-{candidate.AI_Eng_CRL_Rank} Adv-{candidate.Adv_CRL_Rank} Choice - {candidate.current_index}")
  
print("\nFinal Waitlists:")
for program_key, program in programs.items():
    if len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")
    
print("\nFinal Cutoffs:")
for program_key, program in programs.items():
    if len(program.waitlist)>0:
        last_rank = -program.waitlist[0][0]
        print(f"Program {program_key} cutoff: {last_rank}")

In [ ]:
# def load_fcandidates(filename):
#     df = pd.read_csv(filename)
#     filtered_df = df.sort_values('Adv_CRL_Rank')
#     filtered_df = filtered_df[filtered_df['Nationality'] == 4]
#     return {row['RollNo']: Candidate(row['RollNo']
#       ,row['AI_Eng_CRL_Rank']  
#       ,row['AI_Eng_OBC_NCL_Rank']
#       ,row['AI_Eng_SC_Rank']
#       ,row['AI_Eng_ST_Rank']
#       ,row['AI_Eng_EWS_Rank']
#       ,row['AI_Eng_CRL_PD_Rank']
#       ,row['AI_Eng_OBC_NCL_PD_Rank']
#       ,row['AI_Eng_SC_PD_Rank']
#       ,row['AI_Eng_ST_PD_Rank']
#       ,row['AI_Eng_EWS_PD_Rank']
#       ,row['AI_ARC_CRL_Rank']
#       ,row['AI_ARC_OBC_NCL_Rank']
#       ,row['AI_ARC_SC_Rank']
#       ,row['AI_ARC_ST_Rank']
#       ,row['AI_ARC_EWS_Rank']
#       ,row['AI_ARC_CRL_PD_Rank']
#       ,row['AI_ARC_OBC_NCL_PD_Rank']
#       ,row['AI_ARC_SC_PD_Rank']
#       ,row['AI_ARC_ST_PD_Rank']
#       ,row['AI_ARC_EWS_PD_Rank']
#       ,row['AI_Pln_CRL_Rank']
#       ,row['AI_Pln_OBC_NCL_Rank']
#       ,row['AI_Pln_SC_Rank']
#       ,row['AI_Pln_ST_Rank']
#       ,row['AI_Pln_EWS_Rank']
#       ,row['AI_Pln_CRL_PD_Rank']
#       ,row['AI_Pln_OBC_NCL_PD_Rank']
#       ,row['AI_Pln_SC_PD_Rank']
#       ,row['AI_Pln_ST_PD_Rank']
#       ,row['AI_Pln_EWS_PD_Rank']
#       ,row['Adv_CRL_Rank']
#       ,row['Adv_OBC_NCL_Rank']
#       ,row['Adv_SC_Rank']
#       ,row['Adv_ST_Rank']
#       ,row['Adv_EWS_Rank']
#       ,row['Adv_CRL_PD_Rank']
#       ,row['Adv_OBC_NCL_PD_Rank']
#       ,row['Adv_SC_PD_Rank']
#       ,row['Adv_ST_PD_Rank']
#       ,row['Adv_EWS_PD_Rank']
#       ,row['Adv_Prep_SC_Rank']
#       ,row['Adv_Prep_ST_Rank']
#       ,row['Adv_Prep_CRL_PD_Rank']
#       ,row['Adv_Prep_OBC_NCL_PD_Rank']
#       ,row['Adv_Prep_SC_PD_Rank']
#       ,row['Adv_Prep_ST_PD_Rank']
#       ,row['Adv_Prep_EWS_PD_Rank'], row['Adv_DS'], row['Adv_IsPrep']) for index, row in filtered_df.iterrows()}
# foreignCandidates = load_fcandidates(path_candidates)

In [ ]:
# def load_fprograms(filename):
#     df = pd.read_csv(filename)
#     return {(row['Instcd'], row['BrCd'], 'AI', 'OP', 'NO', 'B', 0): Program(row['Instcd'], row['BrCd'], 'AI', 'OP', 'NO', 'B', row['totalseat'], 0) for index, row in df.iterrows()}

# foreignPrograms = load_fprograms(path_seatsFor)

# print("Foreign candidates and seats loaded.")

In [ ]:
# # def load_choices_linear(filename):
# #     df = pd.read_csv(filename)
# #     for index, row in df.iterrows():
# #         roll_no = int(row['Rollno'])
# #         choice_no = row['Optno']
# #         inst_code = row['Instcd']
# #         brcode = row['Brcd']
# #         if roll_no in foreignCandidates.keys():
# #             program_key = (inst_code, brcode, 'AI', 'OP', 'NO', 'B', 0)
# #             program = foreignPrograms.get(program_key)
# #             foreignCandidates[roll_no].choice_list[choice_no-1]=program
# #             foreignCandidates[roll_no].max_index = foreignCandidates[roll_no].max_index+1

# def fprocess_chunk(chunk):
#     local_candidates = {}
#     for index, row in chunk.iterrows():
#         roll_no = int(row['Rollno'])
#         choice_no = row['Optno']
#         v_choice_no = int(row['Optno'])
#         inst_code = row['Instcd']
#         brcode = row['Brcd']
#         if roll_no in foreignCandidates.keys():
#             if roll_no not in local_candidates:
#                 local_candidates[roll_no] = {'choice_list': {}, 'choice_list_mapper': {}, 'max_index': 0}
#             program_key = (inst_code, brcode, 'AI', 'OP', 'NO', 'B', 0)
#             program = foreignPrograms.get(program_key)
#             local_candidates[roll_no]['choice_list'][v_choice_no - 1] = program
#             local_candidates[roll_no]['choice_list_mapper'][v_choice_no - 1] = choice_no
#             local_candidates[roll_no]['max_index'] += 1
#     return local_candidates

# def fmerge_results(results):
#     for result in results:
#         for roll_no, data in result.items():
#             if roll_no in foreignCandidates:
#                 foreignCandidates[roll_no].choice_list.update(data['choice_list'])
#                 foreignCandidates[roll_no].choice_list_mapper.update(data['choice_list_mapper'])
#                 foreignCandidates[roll_no].max_index += data['max_index']

# def fload_choices_parallel(filename, chunk_size=500000):
#     df = pd.read_csv(filename, chunksize=chunk_size)
#     futures = []
#     with ProcessPoolExecutor(max_workers=20) as executor:
#         for chunk in df:
#             futures.append(executor.submit(fprocess_chunk, chunk))
#         results = []
#         for future in as_completed(futures):
#             results.append(future.result())
#     fmerge_results(results)

    
# start_time = time.time()
# fload_choices_parallel(path_choices)   
# print("Foreign choices loaded")
# end_time = time.time()  # Record end time
# elapsed_time = end_time - start_time  # Calculate elapsed time
# print(f"Time taken for loading choices in parallel: {elapsed_time:.2f} seconds")
# print("The tough part is done.")

In [ ]:
# start_time = time.time()
# filtered_candidates = [c for c in foreignCandidates.values()]

# queue = deque(filtered_candidates)

# def getForeignRank(roll_no, program):
#     candidate = foreignCandidates[roll_no]
#     if program.inst_code<200:
#         return candidate.Adv_CRL_Rank
#     elif program.brcode == "4501":
#         return candidate.AI_Pln_CRL_Rank
#     elif program.brcode == "5101" or program.brcode == "5102" or program.brcode == "5103":
#         return candidate.AI_ARC_CRL_Rank
#     else:
#         return candidate.AI_Eng_CRL_Rank

# def f_process_candidates():
#     start_time = time.time()
#     f_allocations = {}
#     while queue:
#         candidate = queue.popleft()
#         if candidate.current_index<=candidate.max_index:
#             preference = foreignCandidates[candidate.roll_no].choice_list[candidate.current_index-1]
#             if preference :
#                 preference = programs[preference.inst_code, preference.brcode, 'AI', 'OP', 'NO', 'B', 0]
#                 foreignPreference = foreignPrograms[preference.inst_code, preference.brcode, 'AI', 'OP', 'NO', 'B', 0]
#                 rk = getForeignRank(candidate.roll_no, preference)
#                 if (len(preference.waitlist)==0 or preference.waitlist[0][0]<-rk) and foreignPreference.add_candidate(rk, candidate.roll_no):
#                     f_allocations[candidate.roll_no] = foreignPreference
#                     if len(foreignPreference.waitlist) > foreignPreference.capacity:
#                         last_rank = -foreignPreference.waitlist[0][0] 
#                         if last_rank > foreignPreference.min_cutoff:
#                             count_same_rank = sum(1 for x in foreignPreference.waitlist if -x[0] == last_rank)

#                             # Check if removing these candidates brings us below capacity
#                             if len(foreignPreference.waitlist) - count_same_rank >= foreignPreference.capacity:
#                                 removed_candidates = []

#                                 # Remove all candidates with the least preferred rank
#                                 while foreignPreference.waitlist and -foreignPreference.waitlist[0][0] == last_rank:
#                                     _, roll_no = heapq.heappop(foreignPreference.waitlist)  # Pop the least preferred candidate
#                                     removed_candidates.append(roll_no)

#                                 # Requeue the removed candidates for further processing
#                                 for roll_no in removed_candidates:
#                                     foreignCandidates[roll_no].current_index += 1  # Move to next preference
#                                     candidate = foreignCandidates[roll_no] 
#                                     queue.append(candidate)  # Requeue for another attempt
#                 else:
#                     foreignCandidates[candidate.roll_no].current_index += 1
#                     queue.append(candidate)  # Add back to the queue if not placed  
                
#     print(f"Time taken: {time.time() - start_time:.2f} seconds")
#     return f_allocations, foreignPrograms

# f_allocations , f_final_programs = f_process_candidates()

# end_time = time.time()  # Record end time
# elapsed_time = end_time - start_time  # Calculate elapsed time
# print(f"Time taken for allocation: {elapsed_time:.2f} seconds")

In [ ]:
# print("Final Allocations:")
# for roll_no, preference in f_allocations.items():
#     candidate = foreignCandidates.get(roll_no)
#     print(f"Candidate {roll_no} is allocated to program {preference.inst_code}'s {preference.brcode} with {preference.quota}, {preference.cat}, {preference.subcat}, {preference.gender} : Mains-{candidate.AI_Eng_CRL_Rank} Adv-{candidate.Adv_CRL_Rank} Choice - {candidate.current_index}")
  
# print("\nFinal Waitlists:")
# for program_key, program in f_final_programs.items():
#     if len(program.waitlist)>0:
#         print(f"Program {program_key} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")
    
# print("\nFinal Cutoffs:")
# for program_key, program in f_final_programs.items():
#     if len(program.waitlist)>0:
#         last_rank = -program.waitlist[0][0]
#         print(f"Program {program_key} cutoff: {last_rank}")

In [ ]:
import csv

unallocated = []

def getPRank(roll_no, program):
    candidate = candidates[roll_no]
    if program.cat == "OP":
        return str(candidate.Adv_Prep_CRL_PD_Rank)+"P"
    elif program.cat == "BC":
        return str(candidate.Adv_Prep_OBC_NCL_PD_Rank)+"P"
    elif program.cat == "EW":
        return str(candidate.Adv_Prep_EWS_PD_Rank)+"P"
    elif program.cat == "SC":
        if program.subcat == "NO":
            return str(candidate.Adv_Prep_SC_Rank)+"P"
        else:
            return str(candidate.Adv_Prep_SC_PD_Rank)+"P"
    elif program.cat == "ST":
        if program.subcat == "NO":
            return str(candidate.Adv_Prep_ST_Rank)+"P"
        else:
            return str(candidate.Adv_Prep_ST_PD_Rank)+"P"
    else:
        return -1

with open(path_output, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No", "Program Code", "Branch Code", "Quota", "Category", "Subcategory", "Gender", "Rank", "Choice Index", "Flag"])
    for roll_no, preference in allocations.items():
        candidate = candidates.get(roll_no)
        if candidate.current_index>candidate.max_index:
            unallocated.append(candidate.roll_no)
        else:
            isPrep="N"
            if preference:
                allocRk=getRank(roll_no, preference)
                if candidate.prep==1 and preference.inst_code<200:
                    isPrep="P"
                    allocRk=getPRank(roll_no, preference)
                writer.writerow([
                    roll_no,
                    preference.inst_code,
                    preference.brcode,
                    preference.quota,
                    preference.cat,
                    preference.subcat,
                    preference.gender,
                    allocRk,
                    candidate.choice_list_mapper[candidate.current_index-1],
                    isPrep
                ])
    for inst_code, students in ds_inst.items():
        for student in students:
            ch=-1
            candidate=candidates[student['roll_no']]
            if candidate.current_index-1 in candidate.choice_list_mapper.keys():
                ch=candidate.choice_list_mapper[candidate.current_index-1]
                writer.writerow([
                        student['roll_no'],
                        inst_code,
                        student['brcode'],
                        "AI",
                        "OP",
                        "NO",
                        "B",
                        student['rank'],
                        ch,
                        "D"
                    ])
            else:
                unallocated.append(candidate.roll_no)
    # for roll_no, preference in f_allocations.items():
    #     candidate = foreignCandidates.get(roll_no)
    #     if candidate.current_index>candidate.max_index:
    #         unallocated.append(candidate.roll_no)
    #     else:
    #         if preference:
    #             writer.writerow([
    #                 roll_no,
    #                 preference.inst_code,
    #                 preference.brcode,
    #                 preference.quota,
    #                 preference.cat,
    #                 preference.subcat,
    #                 preference.gender,
    #                 getForeignRank(roll_no, preference),
    #                 candidate.current_index,
    #                 "F"
    #             ])

print("Allocations generated")

with open(path_error, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No"])
    for roll_no in unallocated:
        writer.writerow([roll_no])

print("Unallocations generated")

In [ ]:
print(reallocs)

In [ ]:
sups={}
totSeats={}
alloSeats={}
for program_key, program in programs.items():
    if program_key[3]+program_key[4] not in alloSeats.keys():
        alloSeats[program_key[3]+program_key[4]]=0
    alloSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
    if program.capacity<len(program.waitlist):
        print(f"Supernumerary in {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {program_key[4]}, {program_key[5]} : {len(program.waitlist)-program.capacity} seats")
        if program_key[3] not in sups.keys():
            sups[program_key[3]]=0
        sups[program_key[3]]+=len(program.waitlist)-program.capacity
        if program_key[3]+program_key[4] not in totSeats.keys():
            totSeats[program_key[3]+program_key[4]]=0
        totSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
    else:
        if program_key[3]+program_key[4] not in totSeats.keys():
            totSeats[program_key[3]+program_key[4]]=0
        totSeats[program_key[3]+program_key[4]]+=program.capacity
        
        
print(sups)
print(alloSeats)
print(totSeats)

In [ ]:
def load_allotment(filename):
    df = pd.read_csv(filename)
    return {(row['Rollno'], row['Optno']): programs[row['Instcd'], row['Brcd'],  row['AllottedQuota'], row['AllottedCat'][:2], row['AllottedCat'][2:], row['AllottedGender'], 0] for index, row in df.iterrows()}
# RoundNo,Rollno,Birth_Cat,Rank,Optno,Instcd,Brcd,AllottedCat,AllottedQuota,AllottedGender,Flag,SupNumReason,WithDraw,RStatus,SStatus,Status,ADate,PIStatus

   # return {(row['RollNO'], row['ChoiceNo']): programs[row['InstCode'], row['Brcode'],  row['AllottedQuota'], row['AllottedCategory'][:2], row['AllottedCategory'][2:], row['AllottedGender'], 0] for index, row in df.iterrows()}
given_all = load_allotment(path_allotment)

for keyo, alloted in given_all.items():    
    roll_no=keyo[0]
    chno=keyo[1]
    if roll_no in candidates.keys():
        if candidates[roll_no].isDS==0:
            if  roll_no in allocations:
                if allocations[roll_no]!=alloted:
                    if allocations[roll_no]:
                        print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but got ({candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]})({allocations[roll_no].inst_code},{allocations[roll_no].brcode},{allocations[roll_no].quota},{allocations[roll_no].cat}{allocations[roll_no].subcat},{allocations[roll_no].gender}) with {candidates[roll_no].AI_Eng_CRL_Rank} and {candidates[roll_no].Adv_CRL_Rank}")
                    else:
                        print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) with {candidates[roll_no].AI_Eng_CRL_Rank} and {candidates[roll_no].Adv_CRL_Rank}")
            else:
                print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but didn't enter")
    # elif not roll_no in foreignCandidates.keys():    
    #     if candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]!=chno:
    #                 print(f"DS {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but got ({candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]})({allocations[roll_no].inst_code},{allocations[roll_no].brcode},{allocations[roll_no].quota},{allocations[roll_no].cat}{allocations[roll_no].subcat},{allocations[roll_no].gender}) with {candidates[roll_no].AI_Eng_CRL_Rank} and {candidates[roll_no].Adv_CRL_Rank}")